# 🔧 Setup

<div style="background-color:#fff3cd; padding:12px; border-left:5px solid #ffa502;">
Run the next two cells to manage setup and imports.
</div>


In [ ]:
import os
import sys
import subprocess

try:
    import google.colab

    print("Running in Google Colab")

    repo_url = "https://github.com/MyoInteract/MyoInteract.git"
    repo_dir = "MyoInteract"

    original_dir = os.getcwd()

    # Clone only if not present
    if not os.path.exists(repo_dir):
        print("Cloning MyoInteract...")
        subprocess.check_call(["git", "clone", repo_url])
    else:
        print("MyoInteract already exists — skipping clone")

    # Install editable
    os.chdir(repo_dir)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", "."])

    # Return to original directory
    os.chdir(original_dir)

    print("Installation complete ✅")

except ImportError:
    print("Not running in Google Colab — assuming local environment.")

In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["MUJOCO_GL"] = "egl"
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags

In [ ]:
import wandb
from myosuite.envs.myo.myouser.hydra_cli import load_config_interactive
from myosuite.envs.myo.myouser.train_jax_ppo import train
from myosuite.envs.myo.myouser.gradio_ui import get_ui, RunState

# 📈 Initialise wandb tracking

<div style="background-color:#ffebee; padding:12px; border-left:5px solid #c62828;">An API key must be provided.</div>

You can find your API key in your [wandb account settings](https://wandb.ai/authorize), or set the `WANDB_API_KEY` environment variable before running the cell below.

If you don't have a wandb account yet, see the [quickstart guide](https://docs.wandb.ai/models/quickstart) for instructions. 

In [ ]:
# An API key must be provided (via prompt below or WANDB_API_KEY env var)
wandb.login()

#### Set the name of your wandb project (optional).

In [ ]:
# Override this to your own chosen value if preferred!
project_name = "myointeract_testing"

## 🖥️ Launch Gradio UI

<div style="background-color:#eef6ff; padding:12px; border-left:5px solid #4a90e2;">

Run the next cell to start the Gradio interface and wait for it to load.

<b>Important:</b> Do not run any subsequent cells until you are satisfied with the configuration and have pressed the <b>Run</b> button.

A URL will appear in the cell output which you can use to access the GUI.

</div>


In [ ]:
state = RunState()
demo = get_ui(project_name, state)
demo.launch(share=True)

# 🚀 Start training

Run the next cell to start training. Results will automatically be logged to wandb!

In [ ]:
cfg = load_config_interactive(state.cfg_overrides, cfg_only=True)
train(cfg)